In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
# ============= CONFIGURAÇÃO DO GOOGLE DRIVE =============
def configurar_google_drive():
    """Monta o Google Drive e configura os caminhos"""

    # Montar Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Configurar caminho para o dataset no Google Drive
    BASE_DIR = "/content/drive/MyDrive/PetImages"

    print(f"Google Drive montado!")
    print(f"Caminho do dataset: {BASE_DIR}")

    # Verificar se as pastas existem
    cat_dir = os.path.join(BASE_DIR, "Cat")
    dog_dir = os.path.join(BASE_DIR, "Dog")

    if not os.path.exists(BASE_DIR):
        print(f"❌ ERRO: Pasta {BASE_DIR} não encontrada!")
        print("📋 Certifique-se de que você:")
        print("   1. Fez upload da pasta PetImages para 'Meu Drive' no Google Drive")
        print("   2. A estrutura está: MyDrive/PetImages/Cat/ e MyDrive/PetImages/Dog/")
        return None

    if not os.path.exists(cat_dir):
        print(f"❌ ERRO: Pasta Cat não encontrada em {cat_dir}")
        return None

    if not os.path.exists(dog_dir):
        print(f"❌ ERRO: Pasta Dog não encontrada em {dog_dir}")
        return None

    print("✅ Pastas encontradas com sucesso!")
    return BASE_DIR

def otimizar_performance_gdrive():
    """Configurações para otimizar performance com Google Drive"""

    # Configurar TensorFlow para usar menos RAM
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Permitir crescimento de memória da GPU
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            print("✅ GPU configurada com crescimento de memória dinâmico")
        except RuntimeError as e:
            print(f"Erro ao configurar GPU: {e}")

    # Configurar para usar cache em disco (acelera leitura do Drive)
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Reduzir logs

    print("⚡ Configurações de performance aplicadas")

# ============= CONFIGURAÇÕES =============
IMG_SIZE = (224, 224)
BATCH_SIZE = 32  # Pode reduzir para 16 se der erro de memória
EPOCHS = 20
LEARNING_RATE = 0.0001

def verificar_dataset(base_dir):
    """Verifica a estrutura do dataset"""
    cat_dir = os.path.join(base_dir, "Cat")
    dog_dir = os.path.join(base_dir, "Dog")

    # Contar arquivos de imagem
    extensoes_validas = ('.png', '.jpg', '.jpeg', '.PNG', '.JPG', '.JPEG')

    num_cats = len([f for f in os.listdir(cat_dir)
                   if f.endswith(extensoes_validas)])
    num_dogs = len([f for f in os.listdir(dog_dir)
                   if f.endswith(extensoes_validas)])

    print(f"📊 Estatísticas do Dataset:")
    print(f"   🐱 Imagens de gatos: {num_cats:,}")
    print(f"   🐶 Imagens de cachorros: {num_dogs:,}")
    print(f"   📁 Total de imagens: {num_cats + num_dogs:,}")

    # Verificar balanceamento
    ratio = min(num_cats, num_dogs) / max(num_cats, num_dogs)
    if ratio < 0.8:
        print(f"⚠️  Dataset desbalanceado (ratio: {ratio:.2f})")
    else:
        print(f"✅ Dataset bem balanceado (ratio: {ratio:.2f})")

    return True

def limpar_dataset_rapido(base_dir):
    """Versão otimizada para remover imagens corrompidas"""
    import cv2

    print("🧹 Limpando dataset (removendo imagens corrompidas)...")

    for class_name in ['Cat', 'Dog']:
        class_dir = os.path.join(base_dir, class_name)
        arquivos = [f for f in os.listdir(class_dir)
                   if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        removed_count = 0
        total_files = len(arquivos)

        print(f"   Verificando {total_files} imagens de {class_name}...")

        for i, filename in enumerate(arquivos):
            if i % 1000 == 0:  # Progresso a cada 1000 imagens
                print(f"   Progresso: {i}/{total_files}")

            filepath = os.path.join(class_dir, filename)
            try:
                # Verificação rápida de tamanho
                if os.path.getsize(filepath) < 1024:  # Menor que 1KB
                    os.remove(filepath)
                    removed_count += 1
                    continue

                # Verificação básica com OpenCV
                img = cv2.imread(filepath)
                if img is None:
                    os.remove(filepath)
                    removed_count += 1

            except Exception:
                try:
                    os.remove(filepath)
                    removed_count += 1
                except:
                    pass  # Arquivo pode ter sido removido por outro processo

        print(f"   ✅ {class_name}: {removed_count} imagens corrompidas removidas")

def criar_data_generators_otimizado(base_dir):
    """Cria geradores de dados otimizados para Google Drive"""

    # Data augmentation para treino - mais conservador para economizar tempo
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1,
        fill_mode='nearest',
        validation_split=0.2
    )

    # Apenas normalização para validação
    val_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2
    )

    # Geradores com configurações otimizadas
    train_generator = train_datagen.flow_from_directory(
        base_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='training',
        seed=42,
        shuffle=True,
        # Importante: usar interpolação mais rápida
        interpolation='bilinear'
    )

    validation_generator = val_datagen.flow_from_directory(
        base_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        subset='validation',
        seed=42,
        shuffle=False,
        interpolation='bilinear'
    )

    return train_generator, validation_generator

def criar_modelo_transfer_learning(base_model_name='ResNet50'):
    """Cria modelo usando transfer learning"""

    print(f"🏗️  Criando modelo base: {base_model_name}")

    # Escolher modelo base
    if base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False,
                          input_shape=(*IMG_SIZE, 3))
    elif base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False,
                             input_shape=(*IMG_SIZE, 3))
    elif base_model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False,
                                input_shape=(*IMG_SIZE, 3))
    else:
        raise ValueError(f"Modelo base '{base_model_name}' não suportado")

    # Congelar as camadas do modelo base inicialmente
    base_model.trainable = False

    # Criar o modelo completo
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),  # Adicionar batch normalization
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])

    return model, base_model

def treinar_modelo(model, train_gen, val_gen, phase="initial"):
    """Treina o modelo com configurações otimizadas"""

    print(f"🚀 Iniciando treinamento - {phase}")

    # Compilar o modelo
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Callbacks otimizados
    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=8,  # Reduzido para economizar tempo
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,
            patience=4,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            f'/content/drive/MyDrive/best_model_{phase}.h5',  # Salvar no Drive
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]

    # Determinar número de épocas
    epochs = EPOCHS if phase == "initial" else EPOCHS // 3

    # Calcular steps por época (importante para grandes datasets)
    steps_per_epoch = train_gen.samples // BATCH_SIZE
    validation_steps = val_gen.samples // BATCH_SIZE

    print(f"   Steps por época: {steps_per_epoch}")
    print(f"   Validation steps: {validation_steps}")

    # Treinar
    history = model.fit(
        train_gen,
        epochs=epochs,
        steps_per_epoch=steps_per_epoch,
        validation_data=val_gen,
        validation_steps=validation_steps,
        callbacks=callbacks,
        verbose=1
    )

    return history

def fine_tuning(model, base_model, train_gen, val_gen):
    """Realiza fine-tuning desbloqueando algumas camadas"""

    print("🔧 Iniciando Fine-tuning...")

    # Descongelar as últimas camadas do modelo base
    base_model.trainable = True

    # Configuração mais conservadora para fine-tuning
    fine_tune_at = int(len(base_model.layers) * 0.7)  # Descongelar últimos 30%

    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    trainable_layers = sum([layer.trainable for layer in model.layers])
    print(f"   🎯 Camadas treináveis após fine-tuning: {trainable_layers}")

    # Recompilar com learning rate muito menor
    model.compile(
        optimizer=Adam(learning_rate=LEARNING_RATE/20),  # Learning rate ainda menor
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Treinar com fine-tuning
    history_ft = treinar_modelo(model, train_gen, val_gen, phase="fine_tuning")

    return history_ft

def plotar_historico(history, history_ft=None):
    """Plota gráficos de loss e accuracy"""

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Loss
    axes[0].plot(history.history['loss'], label='Treino', linewidth=2)
    axes[0].plot(history.history['val_loss'], label='Validação', linewidth=2)
    if history_ft:
        epochs_ft = len(history.history['loss'])
        axes[0].plot(range(epochs_ft, epochs_ft + len(history_ft.history['loss'])),
                    history_ft.history['loss'], label='Treino (Fine-tuning)', linewidth=2)
        axes[0].plot(range(epochs_ft, epochs_ft + len(history_ft.history['val_loss'])),
                    history_ft.history['val_loss'], label='Validação (Fine-tuning)', linewidth=2)
    axes[0].set_title('📉 Loss Durante o Treinamento', fontsize=14)
    axes[0].set_xlabel('Épocas')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Accuracy
    axes[1].plot(history.history['accuracy'], label='Treino', linewidth=2)
    axes[1].plot(history.history['val_accuracy'], label='Validação', linewidth=2)
    if history_ft:
        epochs_ft = len(history.history['accuracy'])
        axes[1].plot(range(epochs_ft, epochs_ft + len(history_ft.history['accuracy'])),
                    history_ft.history['accuracy'], label='Treino (Fine-tuning)', linewidth=2)
        axes[1].plot(range(epochs_ft, epochs_ft + len(history_ft.history['val_accuracy'])),
                    history_ft.history['val_accuracy'], label='Validação (Fine-tuning)', linewidth=2)
    axes[1].set_title('📈 Accuracy Durante o Treinamento', fontsize=14)
    axes[1].set_xlabel('Épocas')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

def avaliar_modelo(model, val_gen):
    """Avalia o modelo final"""

    print("📊 Avaliando modelo final...")

    # Reset do gerador
    val_gen.reset()

    # Predições
    predictions = model.predict(val_gen, verbose=1)
    predicted_classes = (predictions > 0.5).astype(int).flatten()

    # Labels verdadeiros
    true_classes = val_gen.classes[:len(predicted_classes)]  # Garantir mesmo tamanho
    class_labels = list(val_gen.class_indices.keys())

    # Relatório de classificação
    print("\n📋 Relatório de Classificação:")
    print(classification_report(true_classes, predicted_classes, target_names=class_labels))

    # Matriz de confusão
    cm = confusion_matrix(true_classes, predicted_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.title('🎯 Matriz de Confusão')
    plt.ylabel('Real')
    plt.xlabel('Predito')
    plt.show()

    # Accuracy final
    accuracy = np.mean(predicted_classes == true_classes)
    print(f"\n🎉 Accuracy final: {accuracy:.4f} ({accuracy*100:.2f}%)")

    return accuracy

def main():
    """Função principal otimizada para Google Drive"""

    print("🚀 === TRANSFER LEARNING - CATS VS DOGS (Google Drive) ===\n")

    # 1. Configurar Google Drive
    print("📁 Configurando acesso ao Google Drive...")
    base_dir = configurar_google_drive()
    if base_dir is None:
        return None

    # 2. Otimizar performance
    print("\n⚡ Otimizando configurações de performance...")
    otimizar_performance_gdrive()

    # 3. Verificar dataset
    print(f"\n🔍 Verificando dataset em {base_dir}...")
    if not verificar_dataset(base_dir):
        return None

    # 4. Limpar dataset (opcional - pode pular se o dataset já está limpo)
    resposta = input("\n🤔 Deseja limpar o dataset (remover imagens corrompidas)? [s/N]: ")
    if resposta.lower() in ['s', 'sim', 'y', 'yes']:
        limpar_dataset_rapido(base_dir)

    # 5. Criar geradores de dados
    print("\n🔄 Criando geradores de dados...")
    train_gen, val_gen = criar_data_generators_otimizado(base_dir)

    print(f"   Classes: {train_gen.class_indices}")
    print(f"   🏋️  Amostras de treino: {train_gen.samples:,}")
    print(f"   ✅ Amostras de validação: {val_gen.samples:,}")

    # 6. Criar modelo
    modelo_escolhido = 'ResNet50'  # Pode alterar para 'VGG16' ou 'MobileNetV2'
    print(f"\n🏗️  Criando modelo com transfer learning ({modelo_escolhido})...")
    model, base_model = criar_modelo_transfer_learning(modelo_escolhido)

    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])

    print(f"   📊 Parâmetros totais: {total_params:,}")
    print(f"   🎯 Parâmetros treináveis: {trainable_params:,}")

    # 7. Primeira fase de treinamento
    print("\n🚀 === FASE 1: Treinamento Inicial ===")
    history1 = treinar_modelo(model, train_gen, val_gen, "initial")

    # 8. Fine-tuning
    resposta = input("\n🤔 Deseja fazer fine-tuning? [S/n]: ")
    if resposta.lower() not in ['n', 'no', 'nao']:
        print("\n🔧 === FASE 2: Fine-tuning ===")
        history2 = fine_tuning(model, base_model, train_gen, val_gen)
    else:
        history2 = None

    # 9. Plotar resultados
    print("\n📈 Plotando resultados do treinamento...")
    plotar_historico(history1, history2)

    # 10. Avaliar modelo final
    print("\n🎯 === AVALIAÇÃO FINAL ===")
    accuracy = avaliar_modelo(model, val_gen)

    # 11. Salvar modelo final
    modelo_path = '/content/drive/MyDrive/modelo_cats_dogs_final.h5'
    model.save(modelo_path)
    print(f"\n💾 Modelo salvo em: {modelo_path}")
    print(f"🏆 Accuracy final: {accuracy:.4f} ({accuracy*100:.2f}%)")

    return model

In [ ]:
# Executar apenas se o script for executado diretamente
if __name__ == "__main__":
    # Verificar se OpenCV está instalado
    try:
        import cv2

        print("✅ OpenCV encontrado")
    except ImportError:
        print("❌ OpenCV não encontrado. Instalando...")
        !pip install opencv-python
        import cv2

    # Executar função principal
    modelo_final = main()